## Load the input data

In [3]:
file_path='/content/drive/MyDrive/archive (2)/1661-0.txt'
with open(file_path,'r') as f:
  data=f.read()

## Tokenize the text

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [5]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts([data])

In [6]:
len(tokenizer.word_index)

8931

## Create input-output sequences

In [7]:
input_sequences=[]
for i in data.split('\n'):
  tokenized_sentence=tokenizer.texts_to_sequences([i])[0]

  for i in range(1,len(tokenized_sentence)):
    input_sequences.append(tokenized_sentence[:i+1])

In [8]:
input_sequences

[[145, 4790],
 [145, 4790, 1],
 [145, 4790, 1, 1020],
 [145, 4790, 1, 1020, 4],
 [145, 4790, 1, 1020, 4, 128],
 [145, 4790, 1, 1020, 4, 128, 34],
 [145, 4790, 1, 1020, 4, 128, 34, 45],
 [145, 4790, 1, 1020, 4, 128, 34, 45, 611],
 [145, 4790, 1, 1020, 4, 128, 34, 45, 611, 2235],
 [145, 4790, 1, 1020, 4, 128, 34, 45, 611, 2235, 2236],
 [30, 1021],
 [30, 1021, 15],
 [30, 1021, 15, 23],
 [30, 1021, 15, 23, 1],
 [30, 1021, 15, 23, 1, 275],
 [30, 1021, 15, 23, 1, 275, 4],
 [30, 1021, 15, 23, 1, 275, 4, 394],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21, 51],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21, 51, 1676],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21, 51, 1676, 2],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21, 51, 1676, 2, 18],
 [572, 51],
 [572, 51, 3398],
 [572, 51, 3398, 3399],
 [572, 51, 3398, 3399, 13],
 [572, 51, 3398, 3399, 13, 75],
 [572, 51, 3398, 3399, 13, 75, 817],
 [572, 51, 3398, 33

## Get the maximum length from input sequences

In [9]:
max_len=max((len(x)for x in input_sequences))

In [10]:
max_len

20

## Pad the sequences (Here we used Pre-padding)

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
pad_input_sequence=pad_sequences(input_sequences,maxlen=max_len,padding='pre')

In [13]:
pad_input_sequence

array([[   0,    0,    0, ...,    0,  145, 4790],
       [   0,    0,    0, ...,  145, 4790,    1],
       [   0,    0,    0, ..., 4790,    1, 1020],
       ...,
       [   0,    0,    0, ...,    3,  360,   83],
       [   0,    0,    0, ...,  360,   83,  358],
       [   0,    0,    0, ...,   83,  358, 1673]], dtype=int32)

## Create predictors and label

In [14]:
x=pad_input_sequence[:,:-1]
y=pad_input_sequence[:,-1]

In [15]:
x.shape

(101619, 19)

In [16]:
y.shape

(101619,)

## Convert y to one-hot encoded format

In [17]:
from tensorflow.keras.utils import to_categorical
y=to_categorical(y,num_classes=8932)

In [18]:
y.shape

(101619, 8932)

In [19]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [20]:
# Vocabulary size including 0 for padding
vocab_size = len(tokenizer.word_index) + 1

## Import the necessary model libraries

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

## Define the LSTM model

In [22]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(150))
model.add(Dense(vocab_size, activation='softmax'))

## Compile the model

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

## Summary of model

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 19, 100)           893200    
                                                                 
 lstm (LSTM)                 (None, 19, 150)           150600    
                                                                 
 lstm_1 (LSTM)               (None, 19, 150)           180600    
                                                                 
 lstm_2 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 8932)              1348732   
                                                                 
Total params: 2753732 (10.50 MB)
Trainable params: 2753732 (10.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Train the model

In [25]:
model.fit(x,y,epochs=200,callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])

Epoch 1/200
3176/3176 [==============================] - ETA: 0s - loss: 6.4775 - accuracy: 0.0583

3176/3176 [==============================] - 57s 16ms/step - loss: 6.4775 - accuracy: 0.0583
Epoch 2/200
3176/3176 [==============================] - ETA: 0s - loss: 5.8511 - accuracy: 0.0896

3176/3176 [==============================] - 37s 12ms/step - loss: 5.8511 - accuracy: 0.0896
Epoch 3/200
3174/3176 [============================>.] - ETA: 0s - loss: 5.5016 - accuracy: 0.1124

3176/3176 [==============================] - 37s 12ms/step - loss: 5.5018 - accuracy: 0.1125
Epoch 4/200
3175/3176 [============================>.] - ETA: 0s - loss: 5.2632 - accuracy: 0.1281

3176/3176 [==============================] - 38s 12ms/step - loss: 5.2629 - accuracy: 0.1281
Epoch 5/200
3176/3176 [==============================] - ETA: 0s - loss: 5.0593 - accuracy: 0.1410

...

3176/3176 [==============================] - 37s 12ms/step - loss: 0.5079 - accuracy: 0.8600
Epoch 197/200
3176/3176 [==============================] - ETA: 0s - loss: 0.5182 - accuracy: 0.8577

3176/3176 [==============================] - 37s 12ms/step - loss: 0.5182 - accuracy: 0.8577
Epoch 198/200
3176/3176 [==============================] - ETA: 0s - loss: 0.5131 - accuracy: 0.8595

3176/3176 [==============================] - 36s 11ms/step - loss: 0.5131 - accuracy: 0.8595
Epoch 199/200
3175/3176 [============================>.] - ETA: 0s - loss: 0.5136 - accuracy: 0.8586

3176/3176 [==============================] - 38s 12ms/step - loss: 0.5136 - accuracy: 0.8586
Epoch 200/200
3172/3176 [============================>.] - ETA: 0s - loss: 0.5141 - accuracy: 0.8588

3176/3176 [==============================] - 37s 12ms/step - loss: 0.5144 - accuracy: 0.8587


## Save the model

In [26]:
path='/content/drive/MyDrive/'

In [27]:
model.save('second.h5',path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Load the model

In [28]:
from tensorflow.keras.models import load_model

In [29]:
loaded_model=load_model('second.h5')

In [30]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 19, 100)           893200    
                                                                 
 lstm (LSTM)                 (None, 19, 150)           150600    
                                                                 
 lstm_1 (LSTM)               (None, 19, 150)           180600    
                                                                 
 lstm_2 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 8932)              1348732   
                                                                 
Total params: 2753732 (10.50 MB)
Trainable params: 2753732 (10.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Generate the text or predict next words from given input word(s)

In [49]:
import time
import numpy as np

text=' Mr.Sherlock Holmes was'

for i in range(15):
  #tokenize
  token_text=tokenizer.texts_to_sequences([text])[0]
  #padding
  padded_token_text=pad_sequences([token_text],maxlen=19,padding='pre')
  #predict
  pos=np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index==pos:
      text = text + " " + word
      print(text)
      time.sleep(1)

1/1 [==============================] - 0s 22ms/step
 Mr.Sherlock Holmes was following
1/1 [==============================] - 0s 26ms/step
 Mr.Sherlock Holmes was following my
1/1 [==============================] - 0s 32ms/step
 Mr.Sherlock Holmes was following my friend
1/1 [==============================] - 0s 19ms/step
 Mr.Sherlock Holmes was following my friend on
1/1 [==============================] - 0s 19ms/step
 Mr.Sherlock Holmes was following my friend on examining
1/1 [==============================] - 0s 21ms/step
 Mr.Sherlock Holmes was following my friend on examining it
1/1 [==============================] - 0s 21ms/step
 Mr.Sherlock Holmes was following my friend on examining it by
1/1 [==============================] - 0s 19ms/step
 Mr.Sherlock Holmes was following my friend on examining it by a
1/1 [==============================] - 0s 19ms/step
 Mr.Sherlock Holmes was following my friend on examining it by a long
1/1 [==============================] - 0s 19ms/step
 Mr